In [ ]:
from google.colab import userdata
!pip install -qU "langchain-community>=0.2.11" tavily-python
key = userdata.get('TAVILY_API_KEY')
# import subprocess
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = key
# def check_and_install(package_name):
#     try:
#         subprocess.check_output(['pip', 'show', package_name])
#     except subprocess.CalledProcessError:
#         print(f"{package_name} not found, installing...")
#         subprocess.check_output(['pip', 'install', package_name])
#     else:
#         print(f"{package_name} already installed.")

# if 'google.colab' in str(get_ipython()):
#     print('Running on CoLab')

# check_and_install('langchain_community')
# check_and_install('langchain')
# check_and_install('openai')
# check_and_install('python-dotenv')
# check_and_install('tiktoken')
# check_and_install('langchain_openai')
# check_and_install('google-colab')
# !pip install --upgrade langchain
# !pip install langgraph
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in Michigan?")

print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.

tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Michigan City', 'region': 'Indiana', 'country': 'United States of America', 'lat': 41.7075, 'lon': -86.895, 'tz_id': 'America/Chicago', 'localtime_epoch': 1738290960, 'localtime': '2025-01-30 20:36'}, 'current': {'last_updated_epoch': 1738290600, 'last_updated': '2025-01-30 20:30', 'temp_c': 3.3, 'temp_f': 37.9, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 7.8, 'wind_kph': 12.6, 'wind_degree': 164, 'wind_dir': 'SSE', 'pressure_mb': 1011.0, 'pressure_in': 29.86, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 97, 'cloud': 100, 'feelslike_c': 0.1, 'feelslike_f': 32.1, 'windchill_c': -3.9, 'windchill_f': 24.9, 'heatindex_c': 0.9, 'heatindex_f': 33.7, 'dewpoint_c': -0.7, 'dewpoint_f': 30.7, 'vis_km': 11.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 14.8, 'gust_kph': 23.8}}"}, {'url': 'https://www.easeweather.com/north-america/unit

In [ ]:

oai_key = userdata.get('OPENAI_API_KEY')

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = oai_key

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

model_with_tools = model.bind_tools(tools)

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [ ]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Allegan, Michigan?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather Allegan, Michigan'}, 'id': 'call_numG1XyHLLNbZaDDyx67cQ7z', 'type': 'tool_call'}]


In [ ]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [ ]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='06e023ee-b609-427c-9ab6-b3fbd4bd3c57'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 81, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-d9c6e234-8e1f-48c6-9158-4fb709a8992d-0', usage_metadata={'input_tokens': 81, 'output_tokens': 11, 'total_tokens': 92, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [ ]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='be82ea12-3bec-493c-8942-7b2b8bf9e32e'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_QPSy1cMMyz3ZMiKXVUbEHrVE', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 86, 'total_tokens': 109, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-459262a0-646e-4e61-b43b-62bd5999ba9f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_QPSy1cMM

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5hZNAzggLWEmBHOumP0w6Tx8', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 86, 'total_tokens': 109, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b4e81ab3-0a99-429d-86d0-7c082b3d1f97-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_5hZNAzggLWEmBHOumP0w6Tx8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 86, 'output_tokens': 23, 'total_tokens': 

In [ ]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search_results_json with inputs: {'query': 'current weather in San Francisco'}
Done tool: tavily_search_results_json
Tool output was: content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.775, \'lon\': -122.4183, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1738291506, \'localtime\': \'2025-01-30 18:45\'}, \'current\': {\'last_updated_epoch\': 1738290600, \'last_updated\': \'2025-01-30 18:30\', \'temp_c\': 8.3, \'temp_f\': 46.9, \'is_day\': 0, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/116.png\', \'code\': 1003}, \'wind_mph\': 7.2, \'wind_kph\': 11.5, \'wind_degree\': 266, \'wind_dir\': \'W\', \'pressure_mb\': 1024.0, \'pressure_in\': 30.25, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 76, \'cloud\': 75, \'feelslike_c\': 6.3, \'feelslike_f\': 43.4, \'win

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [ ]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I don't know your name. How can I help you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 84, 'total_tokens': 99, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-19f33cd0-db57-48b6-b321-591c02c9eb8b-0', usage_metadata={'input_tokens': 84, 'output_tokens': 15, 'total_tokens': 99, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


In [ ]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I'm sorry, but I don't have access to personal information about users unless you share it with me. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 84, 'total_tokens': 113, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'stop', 'logprobs': None}, id='run-4cb1f499-fd29-4f16-af75-cff887133173-0', usage_metadata={'input_tokens': 84, 'output_tokens': 29, 'total_tokens': 113, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----
